### Binary classification

**Rationale**: Predict the sentiment of customer reviews

In [ ]:
// ML.NET Nuget packages 
#r "nuget:Microsoft.ML,1.4.0"  
    
//Install XPlot package
#r "nuget:XPlot.Plotly,2.0.0"

using Microsoft.ML;
using Microsoft.ML.Data;
using XPlot.Plotly;

In [ ]:
using Microsoft.ML.Data;

public class ProductReview 
{   
    [LoadColumn(0)]
    public bool Sentiment;
       
    [LoadColumn(1)]
    public string Review;     
}

In [ ]:
var dataPath = "../Datasets/Binary-Classification Sentiment/product_reviews.csv";

var mlContext = new MLContext(seed: 1);

IDataView dataView = mlContext.Data.LoadFromTextFile<ProductReview>(dataPath, hasHeader: true, separatorChar: ',');

#### Data exploration

In [ ]:
var rows = mlContext.Data.CreateEnumerable<ProductReview>(dataView, reuseRowObject: false)
                    .Take(10)
                    .ToList();

display(rows);

#### Plotting

In [ ]:
var countPositiveSentiment = dataView.GetColumn<bool>("Sentiment").Count(x => x == true);
var countNegativeSentiment = dataView.GetColumn<bool>("Sentiment").Count(x => x == false);

var values = new List<int> { countPositiveSentiment, countNegativeSentiment};
var labels = new List<string> { "Positive", "Negative" };

var pieChart = Chart.Plot(new Graph.Pie 
{
    labels = labels,
    values = values
});

pieChart

#### Split our data

In [ ]:
var trainTestSplit = mlContext.Data.TrainTestSplit(dataView);

#### Data Transformation

In [ ]:
var dataProcessPipeline = mlContext.Transforms.Text.FeaturizeText("Review")
        .Append(mlContext.Transforms.CopyColumns("Features", "Review"))
        .Append(mlContext.Transforms.NormalizeMinMax("Features", "Features"));

In [ ]:
var trainer = mlContext.BinaryClassification.Trainers.LbfgsLogisticRegression(labelColumnName: "Sentiment", featureColumnName: "Features");

var trainingPipeline = dataProcessPipeline.Append(trainer);

##### Train the model

In [ ]:
ITransformer model = trainingPipeline.Fit(trainTestSplit.TrainSet);

#### Evaluate the model

In [ ]:
var predicitions = model.Transform(trainTestSplit.TestSet);
var metrics = mlContext.BinaryClassification.Evaluate(predicitions, labelColumnName: "Sentiment");

display("Recall:" + " " + metrics.PositiveRecall);
display("Accuracy:" + " " + metrics.Accuracy);
display("F1Score:" + " " + metrics.F1Score); 

#### Save the model

In [ ]:
mlContext.Model.Save(model, trainTestSplit.TrainSet.Schema, "sentimentClassifier.zip");